# **Multi-inference implicit chain of reasoning LM demonstration**

Source: [github](https://github.com/EnronMusk/multi_implicit_cot/tree/main)

In [1]:
import sys

directory_path = r"C:\Users\Luke\multi_implicit_cot"
sys.path.insert(False, directory_path)


### **Dataset Generation**
Here we generate our test and train data. We generate our training data randomly such that it represents less than **1%** of possible observations. We also generate our tokenized data here as well.

---

The format of training and test datasets follow this format:

```
[input 1a] $$$ [input 1b]||[CoT 1a] $$$ [CoT 1b] #### [output 1a] $$$ [output 1b]
[input 2a] $$$ [input 2b]||[CoT 2a] $$$ [CoT 2b] #### [output 2a] $$$ [output 2b]
[input 3a] $$$ [input 3b]||[CoT 3a] $$$ [CoT 3b] #### [output 3a] $$$ [output 3b]
```

In [2]:
from data.data import DatasetGenerator

training_data_size = 777000
test_data_size = 77000

data_path = directory_path
datagen = DatasetGenerator(tokenizer = None, path = data_path, max_len = 1)

datagen.generateTrainDataset(size = training_data_size)
datagen.generateTestDataset(size = test_data_size)